In [1]:
#Drive'a bağlanma
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

/
Mounted at /content/gdrive


In [2]:
#Drive içerisindeki dosyaları gösterme
!ls

bin	 dev   lib32  mydrive			python-apt  srv    usr
boot	 etc   lib64  NGC-DL-CONTAINER-LICENSE	root	    sys    var
content  home  media  opt			run	    tmp
datalab  lib   mnt    proc			sbin	    tools


In [33]:
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import files
import os
import numpy as np
import pandas as pd

In [22]:
%cd /mydrive/
with open('X_train_file.txt', 'r') as FOUT:
    X_train = np.loadtxt(FOUT)

/content/gdrive/My Drive


In [23]:
with open('y_train_file.txt', 'r') as FOUT:
    y_train = np.loadtxt(FOUT)

In [25]:
with open('X_test_file.txt', 'r') as FOUT:
    X_test = np.loadtxt(FOUT)

In [27]:
with open('y_test_file.txt', 'r') as FOUT:
    y_test = np.loadtxt(FOUT)

In [29]:
with open('X_val_file.txt', 'r') as FOUT:
    X_val = np.loadtxt(FOUT)

In [31]:
with open('y_val_file.txt', 'r') as FOUT:
    y_val = np.loadtxt(FOUT)

In [113]:
#Modeli oluşturma
inputs = keras.Input(shape=(None,8192,1))
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
#Binary veri olduğu için activation=sigmoid şeklinde verildi
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [114]:
#Modeli derleme
#Binary sınıf olduğu için kayıp (loss) fonksiyon = binary_crossentropy şeklinde tanımlandı
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              #Ölçüm birimi olarak accuracy baz alındı
              metrics=["accuracy"])

In [115]:
#Validation adımlarının en iyisini kaydetmek ve geri bildirim almak için tanımlandı
#Modeli kontrol eder -- En iyisine kadar bakar ve validation değeri düştüğünde durur
callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="convnet_from_scratch.keras",
 save_best_only=True,
 monitor="val_loss")]

In [ ]:
#Modelin eğitilmesi
validation_data = (X_val,y_val)
history = model.fit(
    X_train,
    y_train,
    #Batch verilerin ne kadarlık boyutlarla taşınacağını belirler -- Modelin eğitimi, epochlar da bu şekilde ilerler
    batch_size=64,
    #Adım sayısı 2 olarak belirlendi
    epochs=2,
    validation_data=validation_data,
    callbacks=callbacks)

In [ ]:
#Model grafiğinin train,val ve bu değerilerin doğruluk değeri ve kayıp fonksiyonuna göre oluşturlması için kullanılır.
import matplotlib.pyplot as plt
accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
#Modelin test edilmesi ve bu test sonucundaki asıl(modelin eğitim sonucundaki) doğruluk değerinin yazdırılması
test_dataset = (X_test,y_test)
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")